# Доработка голосового ассистента

## В домашнем задании вам предстоит выполнить следующие 3 задания:
1) Добавление модели детекции сегментов речи (**3 балла**)
2) Добавление маленькой ASR модели для wake word detection (**3 балла**)
3) добавление Voice Convertion (VC) модели и изменение синтеза речи под ваш голос (**4 балла**)

In [1]:
!pip install torchcodec pydub ffmpeg-python ipywebrtc silero-vad git+https://github.com/openai/whisper.git gdown munch descript-audio-codec -qq
!pip install --upgrade protobuf -qq
!git clone https://github.com/Plachtaa/seed-vc

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Cannot install descript-audio-codec and protobuf==6.33.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
fatal: destination path 'seed-vc' already exists and is not an empty directory.


In [2]:
# 1z0h5pr5rjyJQ34S0g5qNK2bW7QXliL4i
!gdown https://drive.google.com/uc?id=1z0h5pr5rjyJQ34S0g5qNK2bW7QXliL4i
!unzip /content/qwen0.5.zip

Downloading...
From (original): https://drive.google.com/uc?id=1z0h5pr5rjyJQ34S0g5qNK2bW7QXliL4i
From (redirected): https://drive.google.com/uc?id=1z0h5pr5rjyJQ34S0g5qNK2bW7QXliL4i&confirm=t&uuid=59f4334b-5171-4200-b0d5-0ef43006fcd7
To: /content/qwen0.5.zip
100% 788M/788M [00:05<00:00, 142MB/s] 
Archive:  /content/qwen0.5.zip
replace qwen0.5/model.safetensors? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: qwen0.5/model.safetensors  
  inflating: __MACOSX/qwen0.5/._model.safetensors  
  inflating: qwen0.5/LICENSE         
  inflating: __MACOSX/qwen0.5/._LICENSE  
  inflating: qwen0.5/tokenizer_config.json  
  inflating: __MACOSX/qwen0.5/._tokenizer_config.json  
  inflating: qwen0.5/gitattributes   
  inflating: __MACOSX/qwen0.5/._gitattributes  
  inflating: qwen0.5/config.json     
  inflating: __MACOSX/qwen0.5/._config.json  
  inflating: qwen0.5/tokenizer.json  
  inflating: __MACOSX/qwen0.5/._tokenizer.json  
  inflating: qwen0.5/generation_config.json  
  inflating: __MACOS

In [ ]:
import sys
sys.path.append("./seed-vc")

import yaml
import time
from IPython.display import display, HTML, Audio
from ipywebrtc import CameraStream, AudioRecorder, widgets

from google.colab import output
output.enable_custom_widget_manager()

import torch
import torchaudio

from silero_vad import load_silero_vad, get_speech_timestamps

import whisper

from transformers import AutoModelForCausalLM, AutoTokenizer

from hf_utils import load_custom_model_from_hf
from modules.commons import recursive_munch, build_model, load_checkpoint,


Код загрузки voice conversion модели, напрямую к заданиям он не относится

In [ ]:
def load_vc(device):
    # Load model without F0 conditioning
    dit_checkpoint_path, dit_config_path = load_custom_model_from_hf(
        "Plachta/Seed-VC",
        "DiT_seed_v2_uvit_whisper_small_wavenet_bigvgan_pruned.pth",
        "config_dit_mel_seed_uvit_whisper_small_wavenet.yml"
    )

    # Load config
    config = yaml.safe_load(open(dit_config_path, "r"))
    model_params = recursive_munch(config["model_params"])
    model_params.dit_type = 'DiT'

    # Build model
    model = build_model(model_params, stage="DiT")
    hop_length = config["preprocess_params"]["spect_params"]["hop_length"]
    sr = config["preprocess_params"]["sr"]

    # Load checkpoint
    model, _, _, _ = load_checkpoint(
        model,
        None,
        dit_checkpoint_path,
        load_only_params=True,
        ignore_modules=[],
        is_distributed=False,
    )

    # Move to device
    for key in model:
        model[key].eval()
        model[key].to(device)
    model.cfm.estimator.setup_caches(max_batch_size=1, max_seq_length=8192)

    # Load semantic model (Whisper)
    from transformers import AutoFeatureExtractor, WhisperModel
    whisper_name = model_params.speech_tokenizer.name
    whisper_model = WhisperModel.from_pretrained(whisper_name, torch_dtype=torch.float16).to(device)
    del whisper_model.decoder
    whisper_feature_extractor = AutoFeatureExtractor.from_pretrained(whisper_name)

    def semantic_fn(waves_16k):
        ori_inputs = whisper_feature_extractor(
            [waves_16k.squeeze(0).cpu().numpy()],
            return_tensors="pt",
            return_attention_mask=True
        )
        ori_input_features = whisper_model._mask_input_features(
            ori_inputs.input_features,
            attention_mask=ori_inputs.attention_mask
        ).to(device)
        with torch.no_grad():
            ori_outputs = whisper_model.encoder(
                ori_input_features.to(whisper_model.encoder.dtype),
                return_dict=True,
            )
        S_ori = ori_outputs.last_hidden_state.to(torch.float32)
        S_ori = S_ori[:, :waves_16k.size(-1) // 320 + 1]
        return S_ori

    # Load speaker encoder (CAMPPlus)
    from modules.campplus.DTDNN import CAMPPlus
    campplus_ckpt_path = load_custom_model_from_hf(
        "funasr/campplus",
        "campplus_cn_common.bin",
        config_filename=None
    )
    campplus_model = CAMPPlus(feat_dim=80, embedding_size=192)
    campplus_model.load_state_dict(torch.load(campplus_ckpt_path, map_location="cpu"))
    campplus_model.eval()
    campplus_model.to(device)

    # Load vocoder (BigVGAN)
    from modules.bigvgan import bigvgan
    bigvgan_name = model_params.vocoder.name
    bigvgan_model = bigvgan.BigVGAN.from_pretrained(bigvgan_name, use_cuda_kernel=False)
    bigvgan_model.remove_weight_norm()
    bigvgan_model = bigvgan_model.eval().to(device)
    vocoder_fn = bigvgan_model

    # Mel spectrogram function
    from modules.audio import mel_spectrogram
    mel_fn_args = {
        "n_fft": config['preprocess_params']['spect_params']['n_fft'],
        "win_size": config['preprocess_params']['spect_params']['win_length'],
        "hop_size": config['preprocess_params']['spect_params']['hop_length'],
        "num_mels": config['preprocess_params']['spect_params']['n_mels'],
        "sampling_rate": sr,
        "fmin": config['preprocess_params']['spect_params'].get('fmin', 0),
        "fmax": None,
        "center": False
    }
    to_mel = lambda x: mel_spectrogram(x, **mel_fn_args)

    return model, semantic_fn, vocoder_fn, campplus_model, to_mel, sr, hop_length


# Вам нужно модифицировать голосового ассистента, дописав необходимые функции в классе из семинара.

## Задание 1 (3 балла) - добавление модели детекции сегментов с речью
#### Вам нужно написать функцию run_vad в классе голосового ассистента:
1) Загрузите аудио из файла audio_path
2) Ресемплируйте аудио в {self.vad_sample_rate} Hz
3) Детектируйте сегменты с речью в аудио с помощью функции get_speech_timestamps
4) Верните список всех сегментов с речью, найденных в аудио. Если таковых нет - верните пустой список

## Задание 2 (3 балла) - добавление Wake Word Detection модели
#### Вам нужно написать функцию run_wake_word_detection в классе голосового ассистента:
1) Обрежьте или добавьте паддинг из нулей для каждого фрагмента, увеличьте его длину до 30 секунд
2) Посчитайте мел-спектрограмму для каждого фрагмента с помощью функции из библиотеки whisper
3) Получите предсказания с помощью whisper.decode
4) Если в предсказании присутствует wake word - установите self.is_wake_word_detected = True

## Задание 3 (4 балла) - добавление Voice Convertion модели
#### Вам нужно написать функцию run_vc в классе голосового ассистента, реализация основана на seed-vc модели:
1) Извлеките семантические представления (токены) для source и target: ресемплируйте копии аудио в 16 kHz и \
   примените self.vc_semantic_fn к обоим (S_source, S_target).

2) Подготовьте features для эмбеддинга спикера: \
   Вычислите fbank (torchaudio.compliance.kaldi.fbank) на target_16k, c num_mel_bins=80. \
   Вычтите из получившегося тензора среднее по dim=0 (произведите нормализацию). \
   Передайте результат через self.vc_campplus_model, чтобы получить speaker_embed.

3) Сгенерируйте условие для генератора: \
   Cконкатенируйте prompt_condition и cond, \
   выполните self.vc_model.cfm.inference с полученными условиями, длинами и speaker_embed, \
   затем пропустите выход через self.vc_vocoder_fn, получите итоговую волну (vc_wave) и верните её.


In [ ]:
class VoiceAssistant:
    def __init__(
        self,
        device,
        asr_name,
        llm_local_path,
        llm_name,
        tts_language,
        tts_name,
        tts_speaker_name,
        wake_word="",
        enable_vc=False
      ):
        self.device = device

        # ввод аудио
        self.audio_stream = CameraStream(constraints={'audio': True, 'video': False})
        self.recorder = AudioRecorder(stream=self.audio_stream)

        # детекция сегментов с речью
        self.vad = load_silero_vad()
        self.vad_sample_rate = 16000
        self.vad_th = 0.5

        # детекция wake word
        self.is_wake_word_detected = False
        self.wake_word = wake_word
        self.wake_word_asr = whisper.load_model("tiny").to(device)

        # распознавание речи
        self.asr = whisper.load_model(asr_name).to(device)

        # llm агент
        self.llm = AutoModelForCausalLM.from_pretrained(
            llm_local_path,
            dtype="auto",
            device_map="auto"
        )
        self.tokenizer = AutoTokenizer.from_pretrained(llm_name)

        # синтез речи
        self.sample_rate = 48000
        self.speaker = tts_speaker_name
        self.put_accent=True
        self.put_yo=True
        self.put_stress_homo=True
        self.put_yo_homo=True

        tts, _ = torch.hub.load(
            repo_or_dir='snakers4/silero-models',
            model='silero_tts',
            language=tts_language,
            speaker=tts_name
        )
        tts.to(device)
        self.tts = tts

        # voice conversion
        self.enable_vc = enable_vc
        vc_model, vc_semantic_fn, vc_vocoder_fn, vc_campplus_model, vc_to_mel, vc_sr, vc_hop_length = load_vc(device)
        self.vc_model = vc_model
        self.vc_semantic_fn = vc_semantic_fn
        self.vc_vocoder_fn = vc_vocoder_fn
        self.vc_campplus_model = vc_campplus_model
        self.vc_to_mel = vc_to_mel
        self.vc_sr = vc_sr
        self.vc_hop_length = vc_hop_length

    def record(self):
        display(self.recorder)

        save_button = widgets.Button(description="Save Audio")
        output = widgets.Output()

        def on_save_button_clicked(b):
            with output:
                self.recorder.save(self.filename)
                print(f"Audio saved as: {self.filename}")
                display(HTML(f'<a href="{self.filename}" download>Download Audio File</a>'))

        self.filename = f"recording_{time.strftime('%Y%m%d_%H%M%S')}.webm"
        save_button.on_click(on_save_button_clicked)
        display(save_button, output)

    @torch.no_grad()
    def run_asr(self, speech_fragments):
        all_texts = []
        for fragment in speech_fragments:
            fragment = whisper.pad_or_trim(fragment)

            mel = whisper.log_mel_spectrogram(fragment, n_mels=self.asr.dims.n_mels).to(self.asr.device)

            _, probs = self.asr.detect_language(mel)
            # print(f"Detected language: {max(probs, key=probs.get)}")

            options = whisper.DecodingOptions()
            result = whisper.decode(self.asr, mel, options)
            all_texts.append(result[0].text)

        return "".join(t for t in all_texts)

    @torch.no_grad()
    def run_llm(self, recognized_text, max_new_tokens=64):
        prompt = \
f"""
Ты - LLM внутри голосового ассистента, отвечай на вопросы коротко, чтобы модель синтеза речи работала быстро.
Слово, на которое ты активируешься - {self.wake_word}. Если спросят как тебя зовут - {self.wake_word} - твоё имя.

Запрос пользователя: {recognized_text}
Ответ ассисистента: """

        messages = [
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.llm.device)

        generated_ids = self.llm.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return response

    @torch.no_grad()
    def run_tts(self, response_text):
        audio = self.tts.apply_tts(
            text=response_text,
            speaker=self.speaker,
            sample_rate=self.sample_rate,
            put_accent=self.put_accent,
            put_yo=self.put_yo,
            put_stress_homo=self.put_stress_homo,
            put_yo_homo=self.put_yo_homo
        )
        return audio

    @torch.no_grad()
    def run_vad(self, audio_path):
        # Задание 1: Инференс модели детекции сегментов с речью
        #   1) Загрузите аудио из файла audio_path
        #   2) Ресемплируйте аудио в {self.vad_sample_rate} Hz
        #   3) Детектируйте сегменты с речью в аудио с помощью функции get_speech_timestamps
        #   4) Верните список всех сегментов с речью, найденных в аудио. Если таковых нет - верните пустой список

        speech_fragments = []

        # YOUR CODE

        return speech_fragments

    @torch.no_grad()
    def run_wake_word_detection(self, speech_fragments):
        # Задание 2: Инференс модели детекции wake word
        #   1) Обрежьте или добавьте паддинг из нулей для каждого фрагмента, увеличьте его длину до 30 секунд
        #   2) Посчитайте мел-спектрограмму для каждого фрагмента с помощью функции из библиотеки whisper
        #   3) Получите предсказания с помощью whisper.decode
        #   4) Если в предсказании присутствует wake word - установите self.is_wake_word_detected = True

        for fragment in speech_fragments:
            # YOUR CODE
            pass

    @torch.no_grad()
    def run_vc(self, source_audio, target_audio):
        # Задание 3: Инференс Voice Conversion пайплайна
        #   1) Извлеките семантические представления (токены) для source и target: ресемплируйте копии аудио в 16 kHz и
        #      примените self.vc_semantic_fn к обоим (S_source, S_target).
        #   2) Подготовьте features для эмбеддинга спикера:
        #      Вычислите fbank (torchaudio.compliance.kaldi.fbank) на target_16k, c num_mel_bins=80.
        #      Вычтите из получившегося тензора среднее по dim=0 (произведите нормализацию).
        #      Передайте результат через self.vc_campplus_model, чтобы получить speaker_embed.
        #   3) Сгенерируйте условие для генератора:
        #      Cконкатенируйте prompt_condition и cond,
        #      выполните self.vc_model.cfm.inference с полученными условиями, длинами и speaker_embed, \
        #      затем пропустите выход через self.vc_vocoder_fn, получите итоговую волну (vc_wave) и верните её.

        source_audio = source_audio.unsqueeze(0)
        if self.vc_sr != self.sample_rate:
            source_audio = torchaudio.functional.resample(source_audio, orig_freq=self.sample_rate, new_freq=self.vc_sr)

        if self.vc_sr != 16000:
            target_audio = torchaudio.functional.resample(target_audio, orig_freq=16000, new_freq=self.vc_sr)

        source_audio = source_audio.to(self.device).float()
        target_audio = target_audio[:self.vc_sr * 25].to(self.device).float()  # используются первые 25 секунд

        # извлечение семантических представлений
        # YOUR CODE к пункту 1)

        # извлечение мел-спектрограмм
        mel_source = self.vc_to_mel(source_audio)
        mel_target = self.vc_to_mel(target_audio)

        # извлечение эмбеддинга спикера
        # YOUR CODE к пункту 2)

        target_lengths = torch.LongTensor([mel_source.size(2)]).to(self.device)
        target2_lengths = torch.LongTensor([mel_target.size(2)]).to(self.device)

        cond, _, _, _, _ = self.vc_model.length_regulator(S_source, ylens=target_lengths, n_quantizers=3, f0=None)
        prompt_condition, _, _, _, _ = self.vc_model.length_regulator(S_target, ylens=target2_lengths, n_quantizers=3, f0=None)

        # инференс
        cat_condition = # YOUR CODE к пункту 3)

        vc_target = self.vc_model.cfm.inference(
            cat_condition,
            torch.LongTensor([cat_condition.size(1)]).to(self.device),
            mel_target,
            speaker_embed,
            None,
            n_timesteps=10,
            inference_cfg_rate=1.0
        )

        # YOUR CODE к пункту 3)

        return vc_wave.cpu()

    def run_assistant(self):
        # Задание 1 - детекция сегментов речи
        speech_fragments = self.run_vad(self.filename)
        if not len(speech_fragments):
            print("речь не обнаружена")
            return None
        else:
            print("речь обнаружена")

        # Задание 2 - детекция wake word фразы
        if not self.is_wake_word_detected:
            self.run_wake_word_detection(speech_fragments)
            if self.is_wake_word_detected:
                print("wake word задетектированно")
            else:
                print("wake word не задетектированно")

        if self.is_wake_word_detected:
            # распознавание речи
            recognized_text = self.run_asr(speech_fragments)

            # llm обрабатывает запрос
            # Задание 3 - добавление сессионного контекста
            response_text = self.run_llm(recognized_text)

            # синтез ответа llm
            audio_response = self.run_tts(response_text)

            # Задание 4 - добавление модели преобразования голоса
            if self.enable_vc:
                audio_response = self.run_vc(audio_response, speech_fragments[0])

            return audio_response
        return None


Инициализация голосового ассистента

In [15]:
wake_word = "алиса"

assistant = VoiceAssistant(
    device = "cuda",

    asr_name = "base",

    llm_local_path="./qwen0.5",
    llm_name="Qwen/Qwen2.5-0.5B-Instruct",

    tts_language="ru",
    tts_name="v5_ru",
    tts_speaker_name="xenia",

    wake_word = wake_word,

    enable_vc = False
)

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


cfm loaded
length_regulator loaded
Loading weights from nvidia/bigvgan_v2_22khz_80band_256x
Removing weight norm...


#### Пример работы ассистента после выполнения задания 1

In [ ]:
assistant.record()  # запишите аудио без речи

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Button(description='Save Audio', style=ButtonStyle())

Output()

In [ ]:
assistant.run_assistant()  # ассистент ны обрабатывает запрос так как речь не обнаружена

речь не обнаружена


In [ ]:
assistant.record()  # запишите аудио с речью

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Button(description='Save Audio', style=ButtonStyle())

Output()

In [ ]:
assistant.run_assistant()  # речь обнаружена, но wake word не задетектированно: ассистент не обрабатывает запрос

речь обнаружена
wake word не задетектированно


#### Пример работы ассистента после выполнения задания 2

In [ ]:
assistant.record()  # запишите обращение к ассистенту с заданным wake word

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Button(description='Save Audio', style=ButtonStyle())

Output()

In [ ]:
audio = assistant.run_assistant()  # wake word задетектированно: ассистент отвечает
display(Audio(audio.numpy(), rate=assistant.sample_rate))

wake word задетектированно


In [ ]:
assistant.record()  # спросите какое имя у голосового ассистента

AudioRecorder(audio=Audio(value=b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x…

Button(description='Save Audio', style=ButtonStyle())

Output()

In [ ]:
audio = assistant.run_assistant()  # в промпте указано, что wake word также является именем агента: ассистент говорит что его зовут {wake word}
display(Audio(audio.numpy(), rate=assistant.sample_rate))

#### Пример работы ассистента после выполнения задания 3

In [ ]:
assistant.enable_vc = True  # включение Voice Conversion модели в пайплайн

In [ ]:
assistant.record()  # запишите длинный запрос, чтобы ваш голос скопировался более качественно

AudioRecorder(audio=Audio(value=b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x…

Button(description='Save Audio', style=ButtonStyle())

Output()

In [ ]:
audio = assistant.run_assistant()
display(Audio(audio.numpy(), rate=assistant.vc_sr))  # голос ассистента должен стать похожим на ваш из запроса

речь обнаружена
wake word задетектированно


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
100%|██████████| 10/10 [00:00<00:00, 10.83it/s]
